### Thành viên:  
- Huỳnh Minh Thuận - 22110217  
- Trương Minh Thuật - 22110218  
- Nguyễn Phạm Anh Trí - 22110236  
- Nguyễn Minh Trí - 22110235  
- Nguyễn Đình Tiến - 22110230 

### Table of Contents
1. [Introduction](#1.-introduction)  
2. [Data Collection and Ingestion](#2.-data-collection-and-ingestion)  
    2.1 [Data Retrieval Functions and Execution Process](#2.1-data-retrieval-functions-and-execution-process)  
    2.2 [Daily Data Scraping and Storing Strategy](#2.2-daily-data-scraping-and-storing-strategy)
3. [Three-Layer Data Lake Processing](#3.-three-layer-data-lake-processing)  
    3.1 [Bronze Layer Procesing](#3.1-bronze-layer-processing)  
    3.2 [Silver Layer Procesing](#3.2-silver-layer-processing)  
    3.3 [Gold Layer Procesing](#3.3-gold-layer-processing)  
4. [Data Warehouse Storing](#4.-data-warehouse-storing)
5. [Exploratory Data Analysis](#5.-exploratory-data-analysis)
6. [Machine Learning for Recommendation System](#6.-machine-learning-for-recommendation-system)
7. [Application with Streamlit](#7.-application-with-streamlit)

### 1. Introduction  
- Trong thế giới hiện đại ngày nay, âm nhạc là một phần không thể thiếu, nó tạo ra sự kết nối về cảm xúc trong mỗi chúng ta. Nhóm chúng mình quyết định sẽ xây dựng một kiến trúc end-to-end data pipelines bao gồm các công việc như thu thập, cào dữ liệu về nghệ sĩ, bài hát, sau đó xử lý, lưu trữ, phân tích, báo cáo và xây dựng hệ thống đề xuất bài hát dựa theo nhu cầu của người dùng. 

- **Data sources**: Nguồn dữ liệu sẽ được thu thập từ https://kworb.net/itunes/extended.html, bao gồm tên của 15,000 nghệ sĩ hàng đầu được cập nhật hàng ngày. Sau đó, ta sẽ sử dụng **Spotify API** để lấy thông tin về nghệ sĩ, album, bài hát và các đặc điểm của bài hát dựa trên danh sách tên nghệ sĩ đã thu thập được từ trang web Kworb.net.

- **Tools**:
    - **Python**: Ngôn ngữ lập trình chính.
    - **Docker**: Dùng để chạy các container, đảm bảo môi trường nhất quán và có thể mở rộng.
    - **MongoDB**: Được sử dụng làm Database để lưu các dữ liệu thu thập từ việc crawl và request API.
    - **HDFS**: Một phần trong kiến trúc Hadoop, được sử dụng làm Data Lake để lưu trữ dữ liệu để xử lý.
    - **Snowflake**: Kho dữ liệu trên nền tảng đám mây (Cloud-Based Data Warehouse).
    - **PowerBI**: Công cụ hiển thị dữ liệu và cung cấp cái nhìn tổng quan toàn diện.
    - **Airflow**: Một framework sử dụng ngôn ngữ Python để lập lịch và chạy các task.

- **Architecture**:    
![My Image](./images/Architecture.png)
- **Link**: Toàn bộ source code:
*https://github.com/mjngxwnj/Big-Data-for-Music-Recommendation-System*

### 2. Data Collection and Ingestion  
We start by collecting data from **Kworb.net**, which includes **15,000** artist name, then use the **Spotify API** to fetch more music-related details. This data is stored in **MongoDB** for further processing and analysis.

#### 2.1 Data Retrieval Functions and Execution Process
#####   2.1.1 Get data from **Kworb.net**  
- **Step 1**: We get the link from the **Kworb.net** and use pandas to get data from the site and use the read html function (**pandas.read_html(url)**) to read all tables from the returned web page. Function pandas.read_html will return a list of tables available on the web page.  

- **Step 2**: Because the information we need to get is from the first table, we proceed to get the first table with 2 columns: **Pos** corresponding to the artist's position on the rankings table and  **Artist** corresponding to the artist's name.  

- **Step 3**: We save the 2 columns we have retrieved to **MongoDB** to proceed with the next steps.  

##### 2.1.2 Get Information of **Artist**:  
- **Step 1**: We get the artist's name from the **MongoDB** database after filtering out artists with no information. We use the spotipy library to connect the api to spotify and get that artist's information. To connect the api with spotify we need 2 things: client id and client secret to connect  

- **Step 2**: We use the function **sp.search** to get the artist's information. This function returns a dictionary with the artist's information.  

- **Step 3**: We put the information into the corresponding columns and transfer that information back to MongoDB.  
    
##### 2.1.3 Get Information of **Artist's Album** and **Track**:  

- **Step 1**: After obtaining the artist's data, including artist id, we use artist id to get data about the album id and save that data into a list for continued use.  

- **Step 2**: To optimize the number of api calls (avoid overload), we divide the album id list into smaller lists to call the api for each sublist. By using the **spotipy.album** function we can get data for 20 albums and tracks contained in that album in one api call.  

- **Step 3**: We save all the data to the **MongoDB** database.  

##### 2.1.4 Get Information of **Track Feature**:  
- **Step 1**: Using the data from the previous step, we have obtained the track id to get more features of each track.  

- **Step 2**: Also to optimize the number of api calls, we divide the list of tracks into sublists to call the api from spotify. By using the **spotipy.audio_features** function we can get the audio features of 100 tracks in one api call.  

- **Step 3**: We save all the data to the **MongoDB** database.  

![Image](./images/crawl_api.jpg)

#### 2.2 Daily Data Scraping and Storing Strategy
##### 2.2.1 Initial Data Scraping and Storing   
- Sau lần thử scraping data ở ngày đầu, ta thấy được việc crawl 15,000 tên nghệ sĩ và phải mất rất nhiều lượt call **Spotify API** để lấy thông tin (có thể bị block API).  

- Để giải quyết vấn đề này, ta sẽ chia 15,000 tên nghệ sĩ trong 3 ngày, mỗi ngày ta sẽ chỉ lấy 5,000 tên nghệ sĩ để call **Spotify API** và lấy thông tin **artist**, **albums**, **tracks**, **track features** và lưu vào CSV file. Khi toàn bộ 15,000 tên nghệ sĩ được sử dụng để lấy thông tin, ta sẽ load toàn bộ csv file lên MongoDB, và gọi bộ data này là **initial data**.  

- Các hàm call để lấy data sẽ tương tự như trên, và ta sẽ lưu bằng csv trước thay vì MongoDB.  
 
##### 2.2.2 Subsequent Data Scraping and Storing  
- Ở những ngày tiếp theo (sau khi đã có được toàn bộ dữ liệu của 15,000 nghệ sĩ), nếu chúng ta tiếp tục crawl 15,000 tên nghệ sĩ mới ở Kworb sau đó dùng **Spotify API** để lấy thông tin **album**, **track**, chúng ta sẽ gặp vấn đề data duplication, vì các nghệ sĩ nổi tiếng như Taylor Swift thường xuyên nằm trong bảng xếp hạng, dẫn đến việc lặp lại dữ liệu của họ nhiều lần khi crawl hằng ngày.  
- Để tránh khỏi vấn đề này, ta sẽ crawl 15,000 tên nghệ sĩ mới (**new artist names**), sau đó thực hiện **_Left Anti Join_** với 15,000 tên nghệ sĩ cũ (**old artist names**) trong MongoDB để tìm ra được những nghệ sĩ mới (chưa nằm trong bộ dữ liệu), gọi là **new artist names**.  

In [ ]:
def load_daily_artist_name_mongoDB(Execution_date: str):
    with mongoDB_client(username = 'huynhthuan', password = 'password') as client:
        client_operations = mongoDB_operations(client)

        """ Đọc dữ liệu về danh sách tên nghệ sĩ cũ từ MongoDB"""
        old_artist_name_data = client_operations.read_data(database_name = 'music_database', collection_name = 'artist_name_collection')    
        old_artist_name_data = old_artist_name_data[['Artist']]
        old_artist_name_data.rename(columns = {'Artist': 'Old_Artist'}, inplace = True)

        """ Bắt đầu crawl tên nghệ sĩ mới """
        new_artist_name_data = crawl_artist_name(Execution_date)

        """ Thực hiện Left Anti Join để tìm ra các nghệ sĩ có ở new artist names nhưng k có trong 
            old arist names -> daily artist names """
        daily_artist_name_data = pd.merge(old_artist_name_data, new_artist_name_data, left_on = 'Old_Artist', right_on = 'Artist', how = 'right')
        daily_artist_name_data = daily_artist_name_data[daily_artist_name_data['Old_Artist'].isnull()][['Pos', 'Artist', 'Execution_date']]
        daily_artist_name_data = daily_artist_name_data.head(3000)
        
        """ Sau đó ta sẽ load daily arist names vào MongoDB """
        client_operations.insert_data(database_name = 'music_database', collection_name = 'artist_name_collection', data = daily_artist_name_data)

<p style="text-align: center;">
    <img src="./images/leftanti_join_artistname.png" alt="Image">
</p>  

- Chiến lược này giúp chúng ta sẽ chỉ lấy các artist chưa có trong bộ dữ liệu để dùng **Spotify API** và lấy thông tin album track -> Hạn chế duplicates, và giảm thiểu số lượng request API. (Sau khi perform left anti join thì số lượng artist names hằng ngày ~3,000).  

- Tên nghệ sĩ mới sẽ được lưu vào MongoDB, sau đó lấy từ MongoDB để gọi Spotify API và lưu dữ liệu nghệ sĩ, album, track và tiếp tục lưu vào MongoDB. Ta sẽ thêm cột **execute_date** để theo dõi ngày chạy dữ liệu và lấy dữ liệu tương ứng để gọi API (tránh lấy data của những ngày trước đó để gọi **Spotify API**).  

<p style="text-align: center;">
    <img src="./images/daily_crawl_data.png" alt="Image">
</p>  

### 3. Three-Layer Data Lake Processing  
Ta sẽ dùng **HDFS** (**Hadoop Distributed File System**) để lưu các bộ dữ liệu được process và transform, và ta sẽ gọi kho dữ liệu này là **Data Lake**.  

Cấu trúc data lake processing system của chúng ta sẽ thiết kế với 3 lớp chính: **Bronze**, **Silver** and **Gold**. Mỗi lớp đóng vai trò quan trọng khác nhau để lưu và xử lý dữ liệu theo từng cấp cho các bước analysis, reporting và xây dựng mô hình Machine Learning.

#### 3.1 Bronze Layer Processing
Ở giai đoạn này, dữ liệu được sẽ được lấy từ **MongoDB** sau khi gọi **Spotify API** để thu thập thông tin về **artists**, **albums**, **tracks**, và **track features**. Sau đó, ta sẽ apply các schemas được định nghĩa trước vào các bộ dữ liệu để đảm bảo đúng kiểu dữ liệu của từng cột.

Các schema được định nghĩa (**Pyspark Schema**) sẽ có dạng như sau: 

In [ ]:
""" Function for getting schemas. """
def get_schema(table_name: str) -> StructType:
    """ Artist schema. """
    artist_schema = [StructField('Artist_ID',     StringType(), True),
                    StructField('Artist_Name',    StringType(), True),
                    StructField('Genres',         ArrayType(StringType(), True), True),
                    StructField('Followers',      IntegerType(), True),
                    StructField('Popularity',     IntegerType(), True),
                    StructField('Artist_Image',   StringType(), True),
                    StructField('Artist_Type',    StringType(), True),
                    StructField('External_Url',   StringType(), True),
                    StructField('Href',           StringType(), True),
                    StructField('Artist_Uri',     StringType(), True),
                    StructField('Execution_date', DateType(), True)]
    #applying struct type
    artist_schema = StructType(artist_schema)

Tương tự cho các bảng **Album**, **Track** và **Track Feature**.  

Bên cạnh đó, ở quá trình này, ta cũng sẽ áp dụng chiến lược **incremental load** để nạp dữ liệu theo từng ngày dựa theo cột **Execution_date** -> Giảm thiểu lượng data cần xử lý, cần transform và load.  
<p style="text-align: center;">
    <img src="./images/incremental_load.png" alt="Image">
</p>  

Ta có thể thấy chỉ những dữ liệu được crawl từ một ngày cụ thể mới được đọc - xử lý - lưu vào **Data Lake** (Không đọc - xử lý - lưu các dữ liệu crawl ở những ngày trước).

Đây là hàm đọc dữ liệu từ **MongoDB** (sau khi scrape), xử lý đơn giản, áp dụng schema, lọc dữ liệu theo chiến lược **incremental load** bằng tham số đầu vào **Execution_date** và load vào **Data Lake** - **Bronze Layer**.

In [ ]:
""" Applying schemas and loading data from MongoDB into HDFS."""
def bronze_layer_processing(Execution_date: str):
    #get spark Session
    with get_sparkSession(appName = 'Bronze_task_spark') as spark:
        """------------------------ BRONZE ARTIST ------------------------"""
        """ Đọc dữ liệu từ MongoDB """
        artist_data = read_mongoDB(spark, database_name = 'music_database', collection_name = 'artist_collection')

        """ Lọc ra các dữ liệu đã crawl ở ngày hiện tại """
        artist_data = artist_data.filter(artist_data['Execution_date'] == Execution_date)

        """ Bắt đầu xử lý """
        print("Starting bronze preprocessing for artist data...")
        #preprocessing before loading data
        try:
            artist_data = artist_data.withColumn('Genres', split(col('Genres'), ",")) \
                                     .withColumn('Followers', col('Followers').cast('int')) \
                                     .withColumn('Popularity', col('Popularity').cast('int')) \
                                     .withColumn('External_Url', get_json_object(col('External_Url'),'$.spotify')) \
                                     .withColumn('Execution_date', col('Execution_date').cast('date'))
            #reorder columns after reading 
            artist_data = artist_data.select('Artist_ID', 'Artist_Name', 'Genres', 
                                            'Followers', 'Popularity', 'Artist_Image', 
                                            'Artist_Type', 'External_Url', 'Href', 'Artist_Uri', 'Execution_date')
            #applying schema        
            artist_data = spark.createDataFrame(artist_data.rdd, schema = get_schema('artist'))

            print("Finished bronze preprocessing for artist data.")

            #upload data into HDFS
            write_HDFS(spark, data = artist_data, direct = 'bronze_data/bronze_artist', 
                       file_type = 'parquet', mode = "append", partition = 'Execution_date')
        except Exception as e:
            print(f"An error occurred while preprocessing bronze data: {e}")

#### 3.2 Silver Layer Processing  
Ở giai đoạn này, ta sẽ đọc dữ liệu từ **Bronze Layer Data Storage**(dữ liệu được xử lý đơn giản và áp dụng schema để định chuẩn kiểu dữ liệu các cột), sau đó thực hiện các quá trình xử lý dữ liệu, bao gồm:  
- **Drop columns**: drop các cột không cần thiết.  
- **Drop null columns**: drop các dòng chứa giá trị null dựa vào subset column được chọn.  
- **Fill null**: thay thế giá trị null bằng các giá trị cụ thể.  
- **Drop duplicate**: drop các dòng bị trùng lắp dữ liệu dựa vào subset column được chọn.  
- **Handle nested**: xử lý các dòng có cấu trúc nested data.  
- **Rename column**: Đổi tên cột cần thiết.  

Để dễ dàng quản lý và tránh các thao tác xử lý dữ liệu lặp đi lặp lại cho từng bảng, ta sẽ tạo 1 **SilverLayer Class**, đây là lớp sẽ thực hiện các quá trình xử lý dữ liệu nêu trên cho từng bảng dữ liệu.  

Với mỗi bảng cần xử lý, ta chỉ cần áp dụng lớp này và truyền các tham số cần thiết như bảng dữ liệu, danh sách cột cần xóa, cần rename, subset column để drop null,...

In [ ]:
""" Create SilverLayer class to process data in the Silver layer. """
class SilverLayer:
    #init 
    def __init__(self, data: pyspark.sql.DataFrame, 
                 drop_columns: list = None, 
                 drop_null_columns: list = None,
                 fill_nulls_columns: dict = None,
                 duplicate_columns: list = None,
                 nested_columns: list = None,
                 rename_columns: dict = None,
                 ):
        #check valid params
        if data is not None and not isinstance(data, pyspark.sql.DataFrame):
            raise TypeError("data must be a DataFrame!")
        if drop_columns is not None and not isinstance(drop_columns, list):
            raise TypeError("drop_columns must be a list!")
        if drop_null_columns is not None and not isinstance(drop_null_columns, list):
            raise TypeError("drop_null_columns must be a list!")
        if fill_nulls_columns is not None and not isinstance(fill_nulls_columns, dict):
            raise TypeError("handle_nulls must be a dict!")
        if duplicate_columns is not None and not isinstance(duplicate_columns, list):
            raise TypeError("duplicate_columns must be a list!")
        if nested_columns is not None and not isinstance(nested_columns, list):
            raise TypeError("handle_nested must be a list!")
        if rename_columns is not None and not isinstance(rename_columns, dict):
            raise TypeError("rename_columns must be a dict!")
        """Initialize class attributes for data processing."""
        self._data = data
        self._drop_columns = drop_columns
        self._drop_null_columns = drop_null_columns
        self._fill_nulls_columns = fill_nulls_columns
        self._duplicate_columns = duplicate_columns
        self._nested_columns = nested_columns
        self._rename_columns = rename_columns


    """ Method to drop unnecessary columns. """
    def drop(self):
        self._data = self._data.drop(*self._drop_columns)

    """ Method to drop rows based on null values in each column. """
    def drop_null(self):
        self._data = self._data.dropna(subset = self._drop_null_columns, how = "all")

    """ Method to fill null values. """
    def fill_null(self):
        for column_list, value in self._fill_nulls_columns.items():
            self._data = self._data.fillna(value = value, subset = column_list)
    """ Method to rename columns. """
    def rename(self):
        for old_name, new_name in self._rename_columns.items():
            self._data = self._data.withColumnRenamed(old_name, new_name)

    """ Method to handle duplicates. """
    def handle_duplicate(self):
        self._data = self._data.dropDuplicates(self._duplicate_columns)

    """ Method to handle nested. """
    def handle_nested(self):
        for column in self._nested_columns:
            self._data = self._data.withColumn(column, explode_outer(column)) \
                                   .withColumn(column, ltrim(column))
    
    """ Main processing. """
    def process(self) -> pyspark.sql.DataFrame:
        #drop unnecessary columns
        if self._drop_columns:
            self.drop() 

        #drop rows contain null values for each col
        if self._drop_null_columns:
            self.drop_null()

        #fill null values
        if self._fill_nulls_columns:
            self.fill_null()
        
        #handle duplicate rows
        if self._duplicate_columns:
            self.handle_duplicate()

        #handle nested columns 
        if self._nested_columns:
            self.handle_nested()

        #rename columns
        if self._rename_columns:
            self.rename()

        return self._data

Sau khi có được lớp **SilverLayer Class**, ta sẽ tiến hành đọc dữ liệu từ lớp **Bronze Layer**, áp dụng lớp này để xử lý dữ liệu, và nạp dữ liệu vào **Silver Layer Storage**.  

Đây là hàm xử lý dữ liệu ở lớp Silver Layer:

In [ ]:
""" Processing silver artist data. """
def silver_artist_process(spark: SparkSession):
    """ Đọc dữ liệu từ Bronze Layer Storage """
    bronze_artist = read_HDFS(spark, HDFS_dir = "bronze_data/bronze_artist", file_type = 'parquet')

    """ Sử dụng lớp SilverLayer """
    silver_artist = SilverLayer(data = bronze_artist, 
                                drop_columns       = ['Artist_Type', 'Href', 'Artist_Uri', 'Execution_date'],
                                drop_null_columns  = ['Artist_ID'], 
                                fill_nulls_columns = {'Followers': 0,
                                                      'Popularity': 0},
                                duplicate_columns  = ['Artist_ID'],
                                nested_columns     = ['Genres'],
                                rename_columns     = {'Artist_ID': 'id',
                                                      'Artist_Name': 'name',
                                                      'Genres': 'genres',
                                                      'Followers': 'followers',
                                                      'Popularity': 'popularity',
                                                      'Artist_Image': 'link_image',
                                                      'External_Url': 'url'})
    
    """ Gọi phương thức process của SilverLayer class để tiến hành xử lý dữ liệu """
    print("Processing for 'silver_artist' ...")
    silver_artist = silver_artist.process()
    print("Finished processing for 'silver_artist'.")
    """ Upload data (silver data) vào Silver Layer Storage """
    write_HDFS(spark, data = silver_artist, direct = "silver_data/silver_artist", file_type = 'parquet')

Xử lý tương tự cho **bronze_album**, **bronze_track** và **bronze_track_feature**.

#### 3.3 Gold Layer Processing  
Ở bước này, sau khi bộ dữ liệu được xử lý ở giai đoạn **Silver Layer** stage, ta sẽ thực hiện quá trình kết hợp các bảng để tạo ra một lược đồ tuân theo cấu trúc **Snowflake schema**, được chuẩn hóa đến mức cao nhất - **3NF**, trong đó **fact table** sẽ là bảng track, và lược đồ này sẽ được áp dụng để tổ chức dữ liệu trong **Data Warehouse**.  

Đây là lược đồ mà chúng ta mong muốn:  
<p style="text-align: center;">
    <img src="./images/schema.jpg" alt="Image">
</p>  

Trong đó, chúng ta sẽ có 5 bảng tương ướng với 5 bộ dữ liệu:  
- **Fact track**: Đây là bảng chính của bộ dữ liệu, chứa thông tin về toàn bộ bài hát, và được thêm vào các cột khóa ngoại nối đến toàn bộ khóa chính của các bảng dữ liệu khác.  
- **Dim artist**: Đây là bảng dữ liệu về các nghệ sĩ và thông tin tương ứng.
- **Dim album**: Đây là bảng dữ liệu về album, chứa các thông tin như tên, bản quyền, ngày phát hành,...  
- **Dim track feature**: Đây là bảng chứa dữ liệu về các feature của 1 track, bao gồm loudness, mode, tempo,... 
- **Dim genres**: Đây là bảng chứa thông tin về toàn bộ thể loại âm nhạc của các nghệ sĩ, bao gồm pop, rock,...  
- **Dim artist-genres**: Vì mối quan hệ giữa **artist** và **genres** là mối quan hệ **many-to-many** (1 nghệ sĩ có thể có nhiều thể loại âm nhạc, và 1 thể loại âm nhạc cũng có thể thuộc về nhiều nghệ sĩ), bảng **dim artist-genres** là cầu nối giữa 2 **artist** và **genres**.  

Như vậy, ta sẽ lấy bộ dữ liệu đã được làm sạch ở **Silver Layer Storage**, và thực hiện một số quá trình join, aggregation để có được một bộ dữ liệu được tổ chức theo lược đồ mà ta đã chuẩn bị.  

In [ ]:
""" Gold layer. """
def gold_layer_processing():
    with get_sparkSession('Gold_task_spark') as spark:
        #Đọc các dữ liệu từ Silver Layer Storage
        silver_artist = read_HDFS(spark, HDFS_dir = 'silver_data/silver_artist', file_type = 'parquet')
        silver_artist.cache()
        silver_album = read_HDFS(spark, HDFS_dir = 'silver_data/silver_album', file_type = 'parquet')
        silver_album.cache()
        silver_track = read_HDFS(spark, HDFS_dir = 'silver_data/silver_track', file_type = 'parquet')
        silver_track.cache()
        silver_track_feature = read_HDFS(spark, HDFS_dir = 'silver_data/silver_track_feature', file_type = 'parquet')
        silver_track_feature.cache()

        """ Create dim_genres table. """
        dim_genres = silver_artist.select('genres').distinct()
        dim_genres = dim_genres.filter(col('genres').isNotNull())
        #add primary key
        dim_genres = dim_genres.withColumn("id", monotonically_increasing_id()) \
                               .withColumn("id", concat(lit("gns"), col('id')))
        #reorder columns
        dim_genres = dim_genres.select("id", "genres")
        #load data into HDFS
        write_HDFS(spark, data = dim_genres, direct = 'gold_data/dim_genres', file_type = 'parquet')


        """ Create dim_artist table. """
        #just drop genres column and distinct row
        dim_artist = silver_artist.drop('genres').distinct()
        write_HDFS(spark, data = dim_artist, direct = 'gold_data/dim_artist', file_type = 'parquet')


        """ Create dim_artist_genres table. """
        #select necessary columns in artist table
        dim_artist_genres = silver_artist.select('id', 'genres') \
                                         .withColumnRenamed('id', 'artist_id')
        #joining tables to map artist IDs and genre IDs
        dim_genres = read_HDFS(spark, HDFS_dir = 'gold_data/dim_genres', file_type = 'parquet')
        dim_artist_genres = dim_artist_genres.join(dim_genres, on = 'genres', how = 'left') \
                                             .withColumnRenamed('id', 'genres_id')
        #drop genres column
        dim_artist_genres = dim_artist_genres.drop('genres')
        #load data into HDFS
        write_HDFS(spark, data = dim_artist_genres, direct = 'gold_data/dim_artist_genres', file_type = 'parquet')


        """ Create dim_album table. """
        #just drop unnecessary columns 
        dim_album = silver_album.drop('artist', 'artist_id', 'total_tracks', 'release_date_precision')
        #load data into HDFS
        write_HDFS(spark, data = dim_album, direct = 'gold_data/dim_album', file_type = 'parquet')


        """ Create dim_track_feature table. """
        #we don't need to do anything since the dim_track_feature table is complete
        #load data into HDFS
        write_HDFS(spark, data = silver_track_feature, direct = 'gold_data/dim_track_feature', file_type = 'parquet')


        """ Create fact_track table. """
        #drop album name and rename track id column
        fact_track = silver_track.drop('album_name') \
                                 .withColumnRenamed('id', 'track_id')
        #get artist ID from silver album table to create a foreign key for the fact_track table
        silver_album = silver_album.select('id', 'artist_id') \
                                   .withColumnRenamed('id', 'album_id')
        fact_track = fact_track.join(silver_album, on = 'album_id', how = 'inner')
        #reorder columns
        fact_track = fact_track.select('track_id', 'artist_id', 'album_id', 'name', 'track_number', 
                                        'disc_number', 'duration_ms', 'explicit', 'url', 'restriction', 'preview')
        #load data into HDFS
        write_HDFS(spark, data = fact_track, direct = 'gold_data/fact_track', file_type = 'parquet')

Qua các bước join bảng và loại bỏ thêm các cột không cần thiết, đồng thời tạo thêm các cột khóa chính, khóa ngoại cho mỗi bảng, Ta sẽ lưu toàn bộ dữ liệu vào **Gold Layer Storage**. 

### 4. Data Warehouse Storing  
Trong dự án này, kho dữ liệu **Data Warehouse** mà chúng ta chọn sẽ là **Snowflake** - một **Cloud-Based Data Warehouse**. **Snowflake** khá mạnh mẽ cho việc lưu trữ dữ liệu clean, báo cáo và phân tích.   

Sau khi bộ dữ liệu ở lớp **Gold Layer** đã hoàn thành, ta sẽ cần khởi tạo **Database**, **Schema** và **Table** ở **Snowflake Data Warehouse**. Sau đó, ta sẽ tải toàn bộ dữ liệu từ lớp **Gold Layer** vào **Snowflake**. Để đơn giản, ta chỉ cần dùng Spark để đọc dữ liệu đã được lưu ở **Gold Layer Storage** trong **Data Lake**, sau đó load dữ liệu lên Data Warehouse.

In [ ]:
def load_data_Snowflake():
    with get_sparkSession("snowflake_load_data_spark") as spark:
        dim_artist = read_HDFS(spark, HDFS_dir = 'gold_data/dim_artist', file_type = 'parquet')
        write_SnowFlake(spark, data = dim_artist, table_name = 'SPOTIFY_MUSIC_DB.SPOTIFY_MUSIC_SCHEMA.dim_artist')

        dim_genres = read_HDFS(spark, HDFS_dir = 'gold_data/dim_genres', file_type = 'parquet')
        write_SnowFlake(spark, data = dim_genres, table_name = 'SPOTIFY_MUSIC_DB.SPOTIFY_MUSIC_SCHEMA.dim_genres')

        dim_artist_genres = read_HDFS(spark, HDFS_dir = 'gold_data/dim_artist_genres', file_type = 'parquet')
        write_SnowFlake(spark, data = dim_artist_genres, table_name = 'SPOTIFY_MUSIC_DB.SPOTIFY_MUSIC_SCHEMA.dim_artist_genres')

        dim_album = read_HDFS(spark, HDFS_dir = 'gold_data/dim_album', file_type = 'parquet')
        write_SnowFlake(spark, data= dim_album, table_name = 'SPOTIFY_MUSIC_DB.SPOTIFY_MUSIC_SCHEMA.dim_album')

        dim_track_feature = read_HDFS(spark, HDFS_dir = 'gold_data/dim_track_feature', file_type = 'parquet')
        write_SnowFlake(spark, data = dim_track_feature, table_name = 'SPOTIFY_MUSIC_DB.SPOTIFY_MUSIC_SCHEMA.dim_track_feature')

        fact_track = read_HDFS(spark, HDFS_dir = 'gold_data/fact_track', file_type = 'parquet')
        write_SnowFlake(spark, data = fact_track, table_name = 'SPOTIFY_MUSIC_DB.SPOTIFY_MUSIC_SCHEMA.fact_track')

### 5. Exploratory Data Analysis  
Sau khi hoàn thành một hệ thống data pipelines hoàn chỉnh, ta sẽ trực quan hóa và phân tích về bộ dữ liệu của chúng ta. Bộ dữ liệu dùng cho công việc EDA sẽ lấy ở lớp **Silver Layer**, đảm bảo bộ dữ liệu vẫn còn đầy đủ các cột quan trọng để dễ dàng kết hợp bảng và phân tích. 

- Chúng ta sẽ sử nhóm các nghệ sĩ theo độ nổi tiếng của họ và chỉ lấy ra 10 người nổi tiếng nhất và sau đó là sử dụng WordCloud để vẽ ra với WordCloud thì khi tên của người nghệ sĩ nào càng to thì đồng nghĩa với việc người nghệ sĩ đó có độ nổi tiếng cao nhất.  
<p style="text-align: center;">
    <img src="./images/wordcloud_artist.png" alt="Image">
</p>  

- Tiếp theo là chúng ta sẽ vẽ barplot các nghệ sĩ nhưng mà lần này là dựa theo lượt followers của người đó trên nền tảng Spotify và ta thấy được rằng người nghệ sĩ có lượt followers nhiều nhất là 1 người của nước Ấn Độ “Arijit Singh”.  
<p style="text-align: center;">
    <img src="./images/top_followers.png" alt="Image">
</p>  

- Tiếp là chúng ta sẽ nhóm các bài hát dựa trên thể loại của bài hát và sau đó là đếm số lần xuất hiện của mỗi thể loại và chọn ra 5 thể loại mà xuất hiện nhiều nhất trong dataset của chúng ta để thấy được 5 thể loại nổi tiếng nhất và cũng không quá bất ngờ ở đây thì “pop” là thể loại đứng đầu tiên. Nhưng có thể thấy rằng ở vị trí top 5 là “rap” có thể đánh giá rằng rap hiện nay đang là 1 thể loại đang phát triển rất nhanh chóng.  
<p style="text-align: center;">
<img src="./images/popular_genres.png" alt="Image">
</p> 

- Chúng ta sẽ nhóm các tên album name dựa trên độ phổ biến và ở đây ta có thể thấy rằng album “Short n' Sweet” của Sabrina Carpenter là phổ biến nhất hoặc có thể nói là được nhiều người yêu thích nhất.
<p style="text-align: center;">
<img src="./images/popular_albums.png" alt="Image">
</p>  

- Ta sẽ đếm số album mà phát hành trong thời dataset thì ở đây ta sẽ chia ra những năm < 2000 và những năm >= 2000, cụ thể là chia ra làm 2 thế kỉ. Thì có thể đánh giá được rằng thị trường âm nhạc ngày càng phát triển khi mỗi năm là số lượng bài phát hành trở nên nhiều hơn.  
<p style="text-align: center;">
<img src="./images/albums_released.png" alt="Image">
</p>  

- Mặc dù không nằm trong album có độ nổi tiếng cao nhưng album “Abbado: Mozart” lại có số bài hát nhiều nhất trong 1 album với 430 bài.  

![Image](./images/album_with_most_tracks.PNG)

- Bên cạnh đó, với những ngày phát hành của các album, ta sẽ biết được thời gian hoạt động của từng nghệ sĩ. Ta sẽ phân tích và trực quan hóa những nghệ sĩ có thời gian hoạt động dài nhất dựa theo thời gian phát hành album của họ.  
![Image](./images/artist_longevity.png) 

Đây là một số thông tin cơ bản ta có thể hiểu được qua việc phân tích và trực quan hóa về bộ dữ liệu của chúng ta. Toàn bộ source code và những phân tích khác còn lại sẽ nằm trong thư mục **notebook/EDA.ipynb**.